In [1]:
import json

import pytest
from httpretty import httpretty

from rasa_core.actions import action
from rasa_core.actions.action import (
    ActionRestart, UtterAction,
    ActionListen, RemoteAction,
    ActionExecutionRejection)
from rasa_core.domain import Domain
from rasa_core.events import Restarted, SlotSet, UserUtteranceReverted
from rasa_core.trackers import DialogueStateTracker
from rasa_core.utils import EndpointConfig

## source from: conftest.py 

In [24]:
from rasa_core import train, server
from rasa_core.agent import Agent
from rasa_core.channels import CollectingOutputChannel, RestInput, channel
from rasa_core.dispatcher import Dispatcher
from rasa_core.domain import Domain
from rasa_core.interpreter import RegexInterpreter
from rasa_core.nlg import TemplatedNaturalLanguageGenerator
from rasa_core.policies.ensemble import SimplePolicyEnsemble, PolicyEnsemble
from rasa_core.policies.memoization import (
    Policy, MemoizationPolicy, AugmentedMemoizationPolicy)
from rasa_core.processor import MessageProcessor
from rasa_core.slots import Slot
from rasa_core.tracker_store import InMemoryTrackerStore
from rasa_core.trackers import DialogueStateTracker
from rasa_core.utils import zip_folder

DEFAULT_DOMAIN_PATH = "data/test_domains/default_with_slots.yml"

DEFAULT_STORIES_FILE = "data/test_stories/stories_defaultdomain.md"

END_TO_END_STORY_FILE = "data/test_evaluations/end_to_end_story.md"

MOODBOT_MODEL_PATH = "examples/moodbot/models/dialogue"

DEFAULT_ENDPOINTS_FILE = "data/test_endpoints/example_endpoints.yml"

@pytest.fixture(scope="session")
def default_domain():
    return Domain.load(DEFAULT_DOMAIN_PATH)
@pytest.fixture(scope="session")
def default_agent(default_domain):
    agent = Agent(default_domain,
                  policies=[MemoizationPolicy()],
                  interpreter=RegexInterpreter(),
                  tracker_store=InMemoryTrackerStore(default_domain))
    training_data = agent.load_data(DEFAULT_STORIES_FILE)
    agent.train(training_data)
    return agent

@pytest.fixture
def default_dispatcher_collecting(default_nlg):
    bot = CollectingOutputChannel()
    return Dispatcher("my-sender", bot, default_nlg)
@pytest.fixture
def default_nlg(default_domain):
    return TemplatedNaturalLanguageGenerator(default_domain.templates)
@pytest.fixture(scope="session")
def default_agent_path(default_agent, path):
    # path = tmpdir_factory.mktemp("agent").strpath
    default_agent.persist(path)
    return path

In [26]:
domain=default_domain()
print(domain.slots)
agent=default_agent(domain)
default_agent_path(agent, "./model_temp")
nlg=default_nlg(domain)
dispatcher_collecting=default_dispatcher_collecting(nlg)

/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 109.15it/s, # trackers=10]
Processed actions: 0it [00:00, ?it/s, # examples=2]

[<TextSlot(name: None)>]


Processed actions: 49it [00:00, 252.63it/s, # examples=49]


In [22]:
def test_remote_action_logs_events(default_dispatcher_collecting,
                                   default_domain):
    tracker = DialogueStateTracker("default",
                                   default_domain.slots)

    endpoint = EndpointConfig("https://abc.defg/webhooks/actions")
    remote_action = action.RemoteAction("my_action",
                                        endpoint)

    response = {
        "events": [
            {"event": "slot", "value": "rasa", "name": "name"}],
        "responses": [{"text": "test text",
                       "buttons": [{"title": "cheap", "payload": "cheap"}]},
                      {"template": "utter_greet"}]}

    httpretty.register_uri(
        httpretty.POST,
        'https://abc.defg/webhooks/actions',
        body=json.dumps(response))

    httpretty.enable()
    events = remote_action.run(default_dispatcher_collecting,
                               tracker,
                               default_domain)
    httpretty.disable()

    assert (httpretty.latest_requests[-1].path ==
            "/webhooks/actions")

    b = httpretty.latest_requests[-1].body.decode("utf-8")

    assert json.loads(b) == {
        'domain': default_domain.as_dict(),
        'next_action': 'my_action',
        'sender_id': 'default',
        'tracker': {
            'latest_message': {
                'entities': [],
                'intent': {},
                'text': None
            },
            'active_form': {},
            'latest_action_name': None,
            'sender_id': 'default',
            'paused': False,
            'followup_action': 'action_listen',
            'latest_event_time': None,
            'slots': {'name': None},
            'events': [],
            'latest_input_channel': None
        }
    }

    assert events == [SlotSet("name", "rasa")]
    print(events)

    channel = default_dispatcher_collecting.output_channel
    assert channel.messages == [
        {"text": "test text", "recipient_id": "my-sender",
         "buttons": [{"title": "cheap", "payload": "cheap"}]},
        {"text": "hey there None!", "recipient_id": "my-sender"}]

In [23]:
domain=default_domain()
print(domain.slots)
agent=default_agent(domain)
nlg=default_nlg(domain)
dispatcher_collecting=default_dispatcher_collecting(nlg)

test_remote_action_logs_events(dispatcher_collecting, domain)

/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 120.03it/s, # trackers=10]
Processed actions: 0it [00:00, ?it/s, # examples=10]

[<TextSlot(name: None)>]


Processed actions: 49it [00:00, 231.21it/s, # examples=49]


In [28]:
from rasa_core.interpreter import INTENT_MESSAGE_PREFIX
def test_agent_handle_message(default_agent):
    message = INTENT_MESSAGE_PREFIX + 'greet{"name":"Rasa"}'
    result = default_agent.handle_message(message,
                                          sender_id="test_agent_handle_message")
    assert result == [{'recipient_id': 'test_agent_handle_message',
                       'text': 'hey there Rasa!'}]
    
domain=default_domain()
agent=default_agent(domain)
nlg=default_nlg(domain)
dispatcher_collecting=default_dispatcher_collecting(nlg)

test_agent_handle_message(agent)

/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 184.83it/s, # trackers=10]
Processed actions: 49it [00:00, 238.46it/s, # examples=49]


In [29]:
domain.as_dict()

{'actions': ['utter_default', 'utter_greet', 'utter_goodbye'],
 'config': {'store_entities_as_slots': True},
 'entities': ['name'],
 'forms': [],
 'intents': [{'greet': {'use_entities': True}},
  {'default': {'use_entities': True}},
  {'goodbye': {'use_entities': True}},
  {'affirm': {'use_entities': True}},
  {'thank_you': {'use_entities': True}},
  {'change_bank_details': {'use_entities': True}},
  {'simple': {'use_entities': True}},
  {'hello': {'use_entities': True}},
  {'why': {'use_entities': True}},
  {'next_intent': {'use_entities': True}}],
 'slots': {'name': {'auto_fill': True,
   'initial_value': None,
   'type': 'rasa_core.slots.TextSlot'}},
 'templates': {'utter_default': [{'text': 'default message'}],
  'utter_goodbye': [{'text': 'goodbye 😢'}, {'text': 'bye bye 😢'}],
  'utter_greet': [{'text': 'hey there {name}!'}]}}

In [35]:
def disp_events(events):
    # assert events == [SlotSet("test", 4)]
    for ev in events:
        print(str(ev))
    
def test_action(default_dispatcher_collecting,
                                   default_domain, fun_name):
    tracker = DialogueStateTracker("default",
                                   default_domain.slots)

    endpoint = EndpointConfig("http://localhost:5055/webhook")
    remote_action = action.RemoteAction(fun_name, endpoint)

    events = remote_action.run(default_dispatcher_collecting,
                               tracker,
                               default_domain)
    disp_events(events)

    channel = default_dispatcher_collecting.output_channel
    print(channel.messages)

def run_fixture(fun_name, fun):
    domain=default_domain()
    agent=default_agent(domain)
    nlg=default_nlg(domain)
    dispatcher_collecting=default_dispatcher_collecting(nlg)
    fun(dispatcher_collecting, domain, fun_name)

run_fixture("my_custom_action", test_action)

/Users/xiaofeiwu/miniconda3/envs/bigdata/lib/python3.6/site-packages/pykwalify/core.py:99: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  data = yaml.load(stream)
Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 117.78it/s, # trackers=10]
Processed actions: 49it [00:00, 320.46it/s, # examples=49]


SlotSet(key: test, value: 4)
[]


In [37]:
@pytest.fixture
def default_processor(default_domain, default_nlg):
    agent = Agent(default_domain,
                  SimplePolicyEnsemble([AugmentedMemoizationPolicy()]),
                  interpreter=RegexInterpreter())

    training_data = agent.load_data(DEFAULT_STORIES_FILE)
    agent.train(training_data)
    tracker_store = InMemoryTrackerStore(default_domain)
    return MessageProcessor(agent.interpreter,
                            agent.policy_ensemble,
                            default_domain,
                            tracker_store,
                            default_nlg)

In [46]:
from rasa_core.channels import CollectingOutputChannel
from rasa_core.channels import UserMessage
from rasa_core.events import (
    ReminderScheduled, UserUttered, ActionExecuted,
    BotUttered, Restarted)
from rasa_nlu.training_data import Message

def test_message_processor(default_processor):
    out = CollectingOutputChannel()
    default_processor.handle_message(UserMessage('/greet{"name":"Core"}', out))
    assert {'recipient_id': 'default',
            'text': 'hey there Core!'} == out.latest_output()

def test_parsing(default_processor):
    message = Message('/greet{"name": "boy"}')
    parsed = default_processor._parse_message(message)
    assert parsed["intent"]["name"] == 'greet'
    assert parsed["entities"][0]["entity"] == 'name'
    for item in parsed:
        print(item)
        print("\t", parsed[item])
    
processor=default_processor(domain, nlg)
test_message_processor(processor)
test_parsing(processor)

Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 118.18it/s, # trackers=10]
Processed actions: 49it [00:00, 247.07it/s, # examples=49]

text
	 /greet{"name": "boy"}
intent
	 {'name': 'greet', 'confidence': 1.0}
intent_ranking
	 [{'name': 'greet', 'confidence': 1.0}]
entities
	 [{'entity': 'name', 'start': 6, 'end': 21, 'value': 'boy'}]
